# Test name: qa_test_12

##### Reason for test:  
A specific QA check to see whether the model can handle two or more demand points being located at exactly the same latitude and longitude.

##### Parameters

Three supplies - vertically aligned - 150km apart

Sixteen demands just for the middle supply (B) - at bearings 90 and 270 - Various distances away from supplies but layered in two sets of 8 on top of each other.

Capacities - A - supply capable of sustaining half demand, B - zero supply, C - supply capable of sustaining half demand

##### Predicted outcome:
The model will function normally but it will just appear that twice the expected demand is coming from each demand point.



In [5]:
import pandas as pd
import psycopg2
import random
import os
from utilities import generate_point_dict 
from utilities import locate_supplies
from utilities import calculate_distances
dir = os.getcwd()
parent_directory = os.path.split(os.path.split(dir)[0])[0]

An example of what the csv data should look like (see column naming)

In [6]:
pd.read_csv(parent_directory + "/web/data/datasets/data.csv", nrows=2)

,demand,demand_id,demand_lat,demand_lng,demand_name,supply,supply_id,supply_lat,supply_lng,supply_name,duration_min,distance_crowflies_km,distance_route_km
0,11.662019,0,50.920128,-2.670739,Green Ln,35.689500,0,51.749314,-0.240863,Roehyde Way,158.883333,192.761855,235.458
1,11.662019,0,50.920128,-2.670739,Green Ln,28.249443,1,50.815128,-2.273901,Deer Park,48.433333,30.269113,50.560


The method for locating demands will change depending on the scenario and the locate_demands function may be different for each different test.

Changed to only give demand points for the second supply out of the three

In [7]:
def locate_demands(supply_df, num_demands_per_supply, dem_distances_km, dem_bearings_degrees, demand_vol, conn):
    demands = []
    supply_counter = 0
    
    supply_dict = supply_df.to_dict(orient="records")
    this_supply = {"lat":supply_dict[1]["supply_lat"], "lng": supply_dict[1]["supply_lng"]}
    for i in range(num_demands_per_supply):
        distance_km = dem_distances_km[i]
        bearing_degrees = dem_bearings_degrees[i]
        this_demand = generate_point_dict(this_supply, bearing_degrees, distance_km, conn)
        this_demand["demand"] = random.choice(demand_vol)
        this_demand["demand_name"] = "demand" + str(supply_counter + i)
        demands.append(this_demand)
    supply_counter = supply_counter + num_demands_per_supply

    demands_df = pd.DataFrame(demands).reset_index()
    demands_df.columns=["demand_id", "demand", "demand_name", "demand_lat", "demand_lng"]
    return(demands_df)

### Run from here to create a new test data set based on the parameters which can be changed below.

In [8]:
# Naming parameters
output_name = "qa_data_12"

# Supply parameters
num_supplies = 3
sup_distances_km = [0, 150, 300]
sup_bearings_degrees = [0, 0, 0]
supply_vol = [800, 0, 800]

# Demand parameters
num_demands_per_supply = 16
dem_distances_km = [25, 25, 50, 50, 75, 75, 100, 100, 25, 25, 50, 50, 75, 75, 100, 100]
dem_bearings_degrees = [90, 270, 90, 270, 90, 270, 90, 270, 90, 270, 90, 270, 90, 270, 90, 270] # Demands will be taken at these exact degrees for each supply
demand_vol = [100]

# Misc
scale = 1 # Used to scale demand to be a proportion of supply (1 by default)

In [9]:
conn = "host='localhost' dbname='postgres' user='postgres' password=''"
conn = psycopg2.connect(conn)

supply_df = locate_supplies(num_supplies, sup_distances_km, sup_bearings_degrees, supply_vol, conn)
demands_df = locate_demands(supply_df, num_demands_per_supply, dem_distances_km, dem_bearings_degrees, demand_vol, conn)


In [10]:
supply_df

,supply_id,supply_lat,supply_lng,supply,supply_name
0,0,51.501109,-1.242375,800,supply0
1,1,52.849173,-1.242375,0,supply1
2,2,54.196930,-1.242375,800,supply2


In [11]:
demands_df

,demand_id,demand,demand_name,demand_lat,demand_lng
0,0,100,demand0,52.848594,-0.871298
1,1,100,demand1,52.848594,-1.613452
2,2,100,demand2,52.846854,-0.500242
3,3,100,demand3,52.846854,-1.984508
4,4,100,demand4,52.843955,-0.129225
5,5,100,demand5,52.843955,-2.355525
6,6,100,demand6,52.839897,0.241733
7,7,100,demand7,52.839897,-2.726483
8,8,100,demand8,52.848594,-0.871298
9,9,100,demand9,52.848594,-1.613452


In [34]:
# Scale supply capacity so that supply>demand by "scale" (ONLY RUN IF SCALING REQUIRED)
#supply_df["supply"] = supply_df["supply"]* demands_df["demand"].sum()/supply_df["supply"].sum()*scale

In [12]:
# Cartestian product
supply_df["cart"] = 1
demands_df["cart"] = 1
all_combos = demands_df.merge(supply_df)
all_combos = all_combos.drop("cart", axis=1)
all_combos

,demand_id,demand,demand_name,demand_lat,demand_lng,supply_id,supply_lat,supply_lng,supply,supply_name
0,0,100,demand0,52.848594,-0.871298,0,51.501109,-1.242375,800,supply0
1,0,100,demand0,52.848594,-0.871298,1,52.849173,-1.242375,0,supply1
2,0,100,demand0,52.848594,-0.871298,2,54.196930,-1.242375,800,supply2
3,1,100,demand1,52.848594,-1.613452,0,51.501109,-1.242375,800,supply0
4,1,100,demand1,52.848594,-1.613452,1,52.849173,-1.242375,0,supply1
5,1,100,demand1,52.848594,-1.613452,2,54.196930,-1.242375,800,supply2
6,2,100,demand2,52.846854,-0.500242,0,51.501109,-1.242375,800,supply0
7,2,100,demand2,52.846854,-0.500242,1,52.849173,-1.242375,0,supply1
8,2,100,demand2,52.846854,-0.500242,2,54.196930,-1.242375,800,supply2
9,3,100,demand3,52.846854,-1.984508,0,51.501109,-1.242375,800,supply0


In [13]:
# Calculate the crowflies distances for each combination
all_combos = calculate_distances(all_combos, conn)
all_combos.head(10)

,demand_id,demand,demand_name,demand_lat,demand_lng,supply_id,supply_lat,supply_lng,supply,supply_name,duration_min,distance_crowflies_km,distance_route_km
0,0,100,demand0,52.848594,-0.871298,0,51.501109,-1.242375,800,supply0,152.020139,152.020139,152.020139
1,0,100,demand0,52.848594,-0.871298,1,52.849173,-1.242375,0,supply1,24.991992,24.991992,24.991992
2,0,100,demand0,52.848594,-0.871298,2,54.196930,-1.242375,800,supply2,152.019811,152.019811,152.019811
3,1,100,demand1,52.848594,-1.613452,0,51.501109,-1.242375,800,supply0,152.015331,152.015331,152.015331
4,1,100,demand1,52.848594,-1.613452,1,52.849173,-1.242375,0,supply1,24.991209,24.991209,24.991209
5,1,100,demand1,52.848594,-1.613452,2,54.196930,-1.242375,800,supply2,152.015102,152.015102,152.015102
6,2,100,demand2,52.846854,-0.500242,0,51.501109,-1.242375,800,supply0,158.065664,158.065664,158.065664
7,2,100,demand2,52.846854,-0.500242,1,52.849173,-1.242375,0,supply1,49.985150,49.985150,49.985150
8,2,100,demand2,52.846854,-0.500242,2,54.196930,-1.242375,800,supply2,158.065272,158.065272,158.065272
9,3,100,demand3,52.846854,-1.984508,0,51.501109,-1.242375,800,supply0,158.055665,158.055665,158.055665


In [14]:
# Save file to CSV
path = dir + "/qa_data_files/" + str(output_name) + ".csv"
all_combos.to_csv(path, index=False)